# Automatic Invoice Generation

<p style="text-align:center;">
    <img src="images/invoice_ani.gif" alt="invoice-animation" title="Invoice" width="800"><br>
    <center><i>Fig. 1: Invoice Generation [source: <a href="https://www.sage.com/en-gb/blog/invoice-cheat-sheet/">Stacey McIntosh / Sage Advice]</a></i></center>
</p>

# Table of Contents

* [1 Introduction](#Introduction)
* [2 Setup Virtual Environment](#Setup-Virtual-Environment)
* [3 Setup Dependencies](#Setup-Dependencies)
* [4 Create Invoice Template](#Create-Invoice-Template)
* [5 Generate PDF from HTML](#Generate-PDF-from-HTML)
* [6 Create Flask Application](#Create-Flask-Application)
* [7 Make Invoice Dynamic](#Make-Invoice-Dynamic)
* [8 Convert Dynamic Invoice to PDF](#Convert-Dynamic-Invoice-to-PDF)
* [9 Get Client Input](#Get-Client-Input)
* [10 Application Testing](#Application-Testing)
* [11 Challenges](#Challenges)
* [12 Further Steps](#Further-Steps)
* [13 References](#References)

## Introduction

In this project, we aim to generate invoices automatically using Python programming. The program will take invoice details as the input, and thereafter generate and email invoices in PDF format to the respective recipients.

## Setup Virtual Environment

First, we will create a virtual environment `inv-env` and activate it and add it to this notebook as a Python kernel.

In [1]:
!python -m venv inv-env
!.\inv-env\Scripts\activate
!ipython kernel install --user --name=inv-env

Installed kernelspec inv-env in C:\Users\de777\AppData\Roaming\jupyter\kernels\inv-env


Next, we will generate a list of all active kernels to check if `inv-env` is active.

In [2]:
!jupyter kernelspec list

Available kernels:
  f1-env       C:\Users\de777\AppData\Roaming\jupyter\kernels\f1-env
  inv-env      C:\Users\de777\AppData\Roaming\jupyter\kernels\inv-env
  news-env     C:\Users\de777\AppData\Roaming\jupyter\kernels\news-env
  sign-env     C:\Users\de777\AppData\Roaming\jupyter\kernels\sign-env
  steam-env    C:\Users\de777\AppData\Roaming\jupyter\kernels\steam-env
  yt-env       C:\Users\de777\AppData\Roaming\jupyter\kernels\yt-env
  python3      D:\Python\share\jupyter\kernels\python3


We will refresh the notebook and select the new kernel by going to: `Toolbar Menu` $\rightarrow$ `Kernel` $\rightarrow$ `Change kernel` $\rightarrow$ `inv-env`.

## Setup Dependencies

We will be using the following libraries :
- `flask` : provides useful tools & features for easy creation of web applications in Python
- `weasyprint` : helps to create PDF documents from simple HTML pages

So, first we will install `flask`.

In [24]:
!pip3 install flask

In order to install `weasyprint` package on Windows, we need to first install GTK. For that, we need to download the latest GTK3 installer from this [github repository](https://github.com/tschoonj/GTK-for-Windows-Runtime-Environment-Installer/releases), launch it and complete the setup keeping the default options selected. Instructions for setup in details can be found [here](https://doc.courtbouillon.org/weasyprint/stable/first_steps.html).

In [36]:
!pip3 install weasyprint

Now, we will create an `app.py` file in our working directory and add the following imports at the top :

>```py
>from weasyprint import HTML
>from flask import Flask
>```

We will now generate a `requirements.txt` file in our working directory to store information about all dependencies for this project using `pipreqs` library ([link](https://pypi.org/project/pipreqs/)).

In [8]:
#!pip3 install pipreqs
!pipreqs --force --encoding=utf8

INFO: Successfully saved requirements file in E:\Git Repo\Auto-Invoice-Generator\requirements.txt


## Create Invoice Template

We need to have a template before creating PDF invoices. For this, we can either search for an HTML invoice template online or create build one. Here, we will use a simple and clean open-source HTML invoice template provided by Sparksuite ([see demo](https://sparksuite.github.io/simple-html-invoice-template/)). We can further customize it to suit our requirements.

The HTML file can be downloaded from here : https://github.com/sparksuite/simple-html-invoice-template/blob/master/invoice.html


<p style="text-align:center;">
    <img src="images/invoice_template.png" alt="invoice-template" title="Invoice Template" width="500"><br>
    <center><i>Fig. 2: Simple Invoice Template [source: <a href="https://sparksuite.github.io/simple-html-invoice-template/">Wes Cossick / Sparksuite]</a></i></center>
</p>

We will make some modifications to this template. Firstly, we will remove the border and drop-shadow from the `.invoice-box`. Then we will add a footer to the page, and change the logo. And finally, we will save the code in `invoice.html` file in the working directory.

## Generate PDF from HTML

Here we will use `weasyprint` library to convert `invoice.html` to `invoice.pdf` using the following code chunk :

>```py
>html = HTML('invoice.html')
>html.write_pdf('invoice.pdf')
>```

Then, we will run the `app.py` file.

In [ ]:
!python app.py

From the output PDF generated, it can be seen that there is a large margin on both sides of the page as shown :

<p style="text-align:center;">
    <img src="images/extra_margin.jpg" alt="margin" title="Extra Margin in PDF" width="600" style="border:2px solid black"><br>
    <center><i>Fig. 3: Extra Margin in Generated PDF</i></center>
</p>

To avoid this, the following lines can be added between the `<style></style>` tags in the `invoice.html` file as follows :

>`@page {
  size: a4 portrait;
  margin: 0mm 0mm 0mm 0mm;
  counter-increment: page;
}`

Now, on running the same code above, we will see that the margin is gone.

In [45]:
!python app.py

## Create Flask Application

Before converting this into a dynamic template, we will first create a basic Flask application. For this, we will make use of the starter code found on [flask’s website](https://flask.palletsprojects.com/en/latest/quickstart/).

>```py
>import os
>app = Flask(__name__)
>
>@app.route('/')
>app.config['TEMPLATES_AUTO_RELOAD'] = True
>
>def hello_world():
>    return 'Hello, World!'
>
>if __name__ == "__main__":
>   app.run(debug=True, use_reloader=True)
>```

Here, we set `app.config['TEMPLATES_AUTO_RELOAD'] = True` in order to check for modifications of the template source at any instance and render the updated version automatically.  We can now run the `app.py` application :

In [5]:
!python app.py

^C


Now, when we open up `127.0.0.1:5000` in a web browser, we should see : `"Hello, World!"`. To return the `invoice.html` template instead, we will first create a `templates` folder and copy our `invoice.html` file to that. And then, we will replace the `"Hello, World!"` with `render_template('invoice.html')` in the `app.py` file as shown below :

>```py
>from flask import Flask, render_template 
>app = Flask(__name__)
>
>@app.route('/')
>def hello_world():
>    return render_template('invoice.html')
>```

On restarting the flask server, we will see the invoice when we open up `127.0.0.1:5000` in our browser.

## Make Invoice Dynamic

To make the invoice dynamic, we will be using Jinja template variables and loops. 

**About Jinja :** Jinja is a web template engine for the Python. It renders the HTML templates which flask sends to the browser. 

When we call `render_template()`, we will pass in keyword variables which will be accessible in our `invoice.html` template. We can start by updating the “Created” date and modifying the `app.py` as follows :

>```py
>from flask import Flask, render_template
>from datetime import datetime
>
>app = Flask(__name__)
>
>@app.route('/')
>def hello_world():
>   today = datetime.today().strftime("%B %d, %Y")
>   return render_template('invoice.html', date = today)
>```

Here, we are using the `datetime` library to get the current date and then the `strftime` method to format it as: *“Month Date, Year”*. Next, we will update our `invoice.html` template in the `templates` directory by replacing the date with: $\{\{\texttt{date}\}\}$. In this way, when we restart the Flask server, we will be getting a response with the updated date.

Similarly, for other objects like invoice no., from and to addresses etc., we will update the `app.py` file accordingly as:

>```py
>def hello_world():
>    today = datetime.today().strftime("%B %d, %Y")
>    invoice_number = 25678
>    duedate = 'August 1, 2023'
>    from_addr = {
>        'company_name': 'GitHub Inc.',
>        'addr1': '88 Colin P Kelly Jr St',
>        'addr2': 'San Francisco, CA 94107'        
>    }
>    to_addr = {
>        'company_name': 'Stack Overflow Ltd.',
>        'person_name': 'Prashanth Chandrasekar',
>        'person_email': 'prashanthc@example.com'
>    }
>    items = [
>            {
>                'title': 'Website design',
>                'charge': 1000.0                
>            },
>            {
>                'title': 'Hosting (3 months)',
>                'charge': 650.0                
>            },
>            {
>                'title': 'Domain name (1 year)',
>                'charge': 200.0                
>            }
>    ]
>    total = sum([i['charge'] for i in items])
>    return render_template('invoice.html',
>                            date = today,
>                            invoice_number = invoice_number,
>                            duedate = duedate,
>                            from_addr = from_addr,
>                            to_addr = to_addr,
>                            items = items,
>                            total = total                            
>                            )
>```


Next, in `invoice.html` template, we will replace the following :

- invoice number $\rightarrow$ $\{\{\texttt{invoice_number}\}\}$

- due date $\rightarrow$ $\{\{\texttt{duedate}\}\}$

- sender's address
    * company's name $\rightarrow$ $\{\{\texttt{from_addr['company_name']}\}\}$
    * address line 1 $\rightarrow$ $\{\{\texttt{from_addr['addr1']}\}\}$
    * address line 2 $\rightarrow$ $\{\{\texttt{from_addr['addr2']}\}\}$

- receiver's address
    * company's name $\rightarrow$ $\{\{\texttt{to_addr['company_name']}\}\}$
    * person's name $\rightarrow$ $\{\{\texttt{to_addr['person_name']}\}\}$
    * person's email address $\rightarrow$ $\{\{\texttt{to_addr['person_email']}\}\}$

- products
    * item's name $\rightarrow$ $\{\{\texttt{item['title']}\}\}$
    * price $\rightarrow$ $\{\{\texttt{item['charge']}\}\}$

- total price $\rightarrow$ $\{\{\texttt{total}\}\}$

Essentially, we are passing multiple arguments while calling `render_template()` and their values are accessed by enclosing them in double curly braces in the form: `{{variable_name}}`. In Jinja, double curly braces allow evaluation of an expression, variable, or function call and print the result into the template.

We will also loop over the list of items using a for loop as : `${{% for item in items %}}`, and then end the loop using : `{{% endfor %}}`.

Complete final code for `invoice.html`  found [here](templates/invoice.html). After landing on the webpage, right-click and click on “View Page Source” to see the the HTML code.

## Convert Dynamic Invoice to PDF

To generate a PDF and send it to the user, we will use `HTML` function from `weasyprint` and `send_file` function from `flask`. We will save the rendered PDF in a `static` folder and use `send_file` to safely send that PDF to the client. The generated PDF file can be found [here](static/invoice.pdf).

>```py
>from flask import Flask, render_template, send_file
>from datetime import datetime
>from weasyprint import HTML
>
># --snip-- #
>
>def hello_world():
>    # --snip-- #
>    rendered = render_template('invoice.html',
>                                date = today,
>                                from_addr = from_addr,
>                                to_addr = to_addr,
>                                items = items,
>                                total = total,
>                                invoice_number = invoice_number,
>                                duedate = duedate)
>    html = HTML(string=rendered)
>    rendered_pdf = html.write_pdf('./static/invoice.pdf')
>    return send_file('./static/invoice.pdf')
>```

Here, instead of returning the rendered template, we are assigning it to the `rendered` variable which is then passed to `HTML` function and saved under `html` variable. Next, we are writing the PDF output to a file and using the `send_file` function to send that PDF to the client.

To skip writing the PDF to disk, we will use the `io` library to render it in memory and send it directly to the client. For that, we don't need to pass any arguments to `write_pdf()`. Instead, we will pass our bytes to the `io.BytesIO()` function and pass that to the `send_file` function. `io.BytesIO()` converts regular bytes to behave in a similar way to a file object. It also provides us with a `.read()` method, which is a requirement of file objects in Flask and is otherwise absent from our `rendered_pdf`.

>```py
># --snip-- #
>import io
>
># --snip-- #
>def hello_world():
>    # --snip-- #
>    html = HTML(string=rendered)
>    print(rendered)
>    rendered_pdf = html.write_pdf()
>    return send_file(io.BytesIO(rendered_pdf), download_name='invoice.pdf')
>```

Now, on running the `app.py` file again and opening up `127.0.0.1:5000` in our browser, we will get a PDF response back.

<p style="text-align:center;">
    <img src="images/response_pdf.png" alt="pdf-response" title="PDF Response" width="600" style="border:2px solid black"><br>
    <center><i>Fig. 4: Generated PDF Response</i></center>
</p>

## Get Client Input

To collect invoice information from the client/user, we will use a basic API, where we will take input in JSON format and
return the output as a PDF file. We will set up the structure of expected input for our API and assign it to a `default_data` dictionary, to have some default data to use in case the user-supplied input is incomplete.

>```py
>default_data =
>{
>    'invoice_number': 25678,
>    'duedate': 'August 1, 2023',
>    'from_addr': {
>            'company_name': 'GitHub Inc.',
>            'addr1': '88 Colin P Kelly Jr St',
>            'addr2': 'San Francisco, CA 94107'            
>        },
>        'to_addr': {
>            'company_name': 'Stack Overflow Ltd.',
>            'person_name': 'Prashanth Chandrasekar',
>            'person_email': 'prashanthc@example.com'            
>        },
>        'items': [{
>                'title': 'Website Designing',
>                'charge': 1000.0
>            },
>            {
>                'title': 'Hosting (3 months)',
>                'charge': 650.0
>            },
>            {
>                'title': 'Domain Name (1 year)',
>                'charge': 200.0
>    }]
>}
>```

Next, we will specify `GET` and `POST` requests in our route (`@app.route`) so that it responds to those type of requests : 

>```py
>@app.route('/', methods=['GET', 'POST'])
>```

In order to access `POST` JSON data in Flask, we will use `request.get_json()` method. If the request is of `GET` type, `request.get_json()` will return `None` and so `posted_data` will be equal to `{}`. This is implemented as follows :

>```py
>from flask import request
># -- snip-- #
>
>def hello_world():
>    # --snip -- #
>    posted_json = request.get_json() or {}
>```

Next, we will use `.get()` method to get data from the `posted_data` dictionary based on respective keys of objects in the invoice. In case a dictionary key is absent or has empty value, `.get()` method will return a default value from `default_data` dictionary that we have defined before. The method for duedate will look like :

>```py
>duedate = posted_data.get('duedate', default_data['duedate'])
>```

Accordingly, we will modify the rest of our `app.py` file data and the final code can be found [here](app.py).

Lastly, we will update our `requirements.txt` file to capture all the dependencies used in `app.py` file :

In [14]:
!pipreqs --force --encoding=utf8

INFO: Successfully saved requirements file in E:\Git Repo\Auto-Invoice-Generator\requirements.txt


## Application Testing

To check if it works, we will use the `requests` library to open up the URL and save the data in a local PDF file. For this, we will create a new file named `app_test.py` ([click to view](app_test.py)) and add the following code to it :

>```py
>import requests
>url = 'http://127.0.0.1:5000/'
>data = {
>    'invoice_number': 15665,
>    'duedate': 'December 1, 2022',
>    'from_addr': {
>        'company_name': 'Python Projects',
>        'addr1': '18 Mt Alexander Rd, Flemington',
>        'addr2': 'Melbourne, VIC 3032'
>    },
>    'to_addr': {
>        'company_name': 'My Company Inc.',
>        'person_name': 'Tapojoy De',
>        'person_email': 'tdjoy@example.com'
>    },
>    'items': [{
>        'title': 'Brochure Designing',
>        'charge': 900.0
>    }, {
>        'title': 'Hosting (6 months)',
>        'charge': 550.0
>    }, {
>        'title': 'Domain name (1 year)',
>        'charge': 200.0
>    }]
>}
>
>html = requests.post(url, json=data)
>
>with open('invoice_test.pdf', 'wb') as f:
>    f.write(html.content)
>```

When we run this file, we will have a file named `invoice_test.pdf` in our current working directory ([click to view](invoice_test.pdf)).

In [13]:
!python app_test.py

^C


<p style="text-align:center;">
    <img src="images/test_output_pdf.png" alt="pdf-output" title="PDF Output" width="500" style="border:2px solid black"><br>
    <center><i>Fig. 5: Invoice PDF Test Output</i></center>
</p>

## Challenges

- The installation of `weasyprint` library might be a challenging task as it has different dependencies based on the system being used. Often, issues arise due to missing packages on the system. One can always refer to their official documentation ([link](https://doc.courtbouillon.org/weasyprint/stable/first_steps.html)) or search Stackoverflow, Google etc. for possible fixes.


- Rendering and sending the file directly to the user as a PDF without saving it to disk can become a bottleneck in some cases, especially in bigger applications. In such cases, we will have to use a task scheduler like Celery package ([link](https://docs.celeryq.dev/en/stable/getting-started/first-steps-with-celery.html#first-steps)) to render the PDF in the background and then send it to the client.


- Our current application can be DOSed (Denial of Service) easily as PDF generation takes time, which might put heavy load on the server causing it not respond to user requests. To help mitigate the risk of DOS, we can run the PDF generation asynchronously using a task queue like Celery.

## Further Steps

- We can extend this project by implementing email functionality. For example, we can design it such that if we pass in an email, the application will render and send the PDF to the passed-in email id.


- Applying the methods outlined in this project, we can further host this application on Heroku or DigitalOcean or any Virtual Private Server.

***

## References

- Pallets. 2022. _Command Line Interface_. Flask Documentation. [URL](https://flask.palletsprojects.com/en/latest/cli/)


- Grinberg, Miguel. 2020. _How to Run a Flask Application_. Twilio Blog. [URL](https://www.twilio.com/blog/how-run-flask-application)


- Refsnes Data. 2022. _HTML Element Reference_. W3Schools. [URL](https://www.w3schools.com/tags/)


- Kravcenko, Vadim. 2021. _Pipreqs Package Documentation_. Python Package Index (PyPI). [URL](https://pypi.org/project/pipreqs/)